# Evaluación Parcial 2 Deep Learning

- Profesor: Felipe Zambrano

### Instrucciones generales

- Esta prohibido compartir las respuestas con otros grupos.
- Indicios de copia serán penalizados con la nota mínima.
- Cualquier duda fuera del horario de clases se debe enviar por el AVA. Mensajes al docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente, si utiliza material extra debe citarlo.

### Sobre la entrega
  * Antes de comenzar a realizar la evaluación, guarde una copia de este cuadernillo en su Google Drive (`Archivo > Guardar una copia en Drive`). No hacer esto puede implicar que su trabajo no se guarde correctamente.
  *  Debe descargar este cuardenillo de Google Colab como un archivo `.ipynb` (`Archivo > Descargar > Descargar .ipynb`). No se revisarán otros formatos de entrega. Recuerde guardar su desarrollo antes de descargar el archivo correspondiente.
  *  El archivo debe ser reproducible, o sea si cargo el archivo en Google Colab y ejecuto todas las celdas, debe reproducir los resultados que usted obtuvo.
  *  Se debe subir el archivo `.ipynb` descargado al AVA en el apartado Experiencia de Aprendizaje 2: Redes Convolucionales -> Evaluación Parcial EA2.
  *  La fecha límite de entrega es el **sábado 14 de junio hasta las 10 am**. Por cada hora de retraso, se descontará un punto de la nota final. Ejemplo: Si entrega entre 10:00 y 10:59, la nota máxima será 6.0. Si entrega entre 11:00 y 11:59, la nota máxima será 5.0, y así sucesivamente.
  *  Ante cualquier sospecha de copia, reprobarán automáticamente el curso con nota mínima (1.0) los involucrados. La discusión entre pares siempre es beneficiosa y fomenta diálogos interesantes, pero cada grupo debe desarrollar su propio trabajo.
  * El cálculo de la nota de la evaluación esta dado por: $\text{Nota}=1+{\sum_{i=1}^7\text{PuntajePregunta}_i \over 7}$


### Grupo Nº
> POR FAVOR AGREGAR EL NÚMERO DE GRUPO

### Integrantes del grupo

> POR FAVOR AGREGAR TODOS LOS NOMBRES DE LOS INTEGRANTES



### Sobre el dataset

Para esta evaluación, cada grupo deberá trabajar con el dataset que contiene imágenes de personajes de Los Simpson. Deberán seleccionar 10 personajes diferentes (categorías/clases) para construir su modelo de clasificación de imágenes con una red neuronal convolucional.

Se recomienda guardar el mejor modelo con `ModelCheckpoint` y usar `TensorBoard` para visualizar la evolución de la función de pérdida y precisión tanto en el conjunto de entrenamiento como en el de validación a lo largo de la épocas.

In [9]:
# 1. Cargar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

In [13]:
# 2. Cargar el dataset con las imágenes de Los Simpson
dataset_path = 'C:/Users/tiare/OneDrive/Escritorio/DeepLearning/DeepLearning2/simpsons_1'

# hacemos primera revisión de los archivos
files = os.listdir(dataset_path)
print(f"Total de archivos en el dataset: {len(files)}")

#Hacemos una revisión de las carpetas
folders = [f for f in files if os.path.isdir(os.path.join(dataset_path, f))]
print(f"Total de carpetas (categorías): {len(folders)}")
# Revisamos las primeras 5 carpetas
print("Primeras 5 carpetas (categorías):", folders[:5])

#creamos un diccionario para almacenar las imágenes y sus etiquetas
images = []
labels = []

# Recorremos las carpetas y cargamos las imágenes
for folder in folders:
    folder_path = os.path.join(dataset_path, folder)
    for file in os.listdir(folder_path):
        if file.endswith('.png') or file.endswith('.jpg'):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))  # Redimensionamos las imágenes a 64x64
                images.append(img)
                labels.append(folder)


Total de archivos en el dataset: 1
Total de carpetas (categorías): 1
Primeras 5 carpetas (categorías): ['agnes_skinner']


In [14]:
#Tomamos cuantos ejemplos tenemos por categoría
category_counts = pd.Series(labels).value_counts()
print("Cantidad de ejemplos por categoría:")
print(category_counts)

Cantidad de ejemplos por categoría:
agnes_skinner    42
Name: count, dtype: int64


In [15]:
#Tenemos que averiguar si las clases están balanceadas
balanced = all(count == category_counts.iloc[0] for count in category_counts)
print(f"Las clases están balanceadas: {balanced}")


El dataset está balanceado: True


**1.** **[6 pts]** Describa las características del dataset:
- Personajes seleccionados (Categorías)
- Tamaño de las imágenes
- Tamaño del dataset
- Tamaño del conjunto de entrenamiento
- Tamaño del conjunto de test
- Indique si las clases están balanceadas


In [ ]:
# Responda aquí:

**2.** **[6 pts]** Genere una previsualización de 10 imágenes por categoría

In [ ]:
# Responda aquí:

**3.** **[6 pts]** Separe el dataset en un conjunto de entrenamiento y test, y posteriormente normalice los datos

In [ ]:
# Responda aquí:

**4.** **[6 pts]** Defina la arquitectura de la red neuronal convolucional con el detalle de cada capa:
- Especifique el tamaño de las imágenes de entrada con sus respectivos canales
- Especifique para cada capa convolucional el número de filtros o canales de la capa, el tamaño del filtro o kernel, el stride, el padding y la función de activación utilizada
- Especifique el tipo de Pooling junto con el tamaño del filtro y el stride cada vez que se utilice
- Especifique el tamaño del vector (flatten) o número de neuronas que ingresan a la MLP
- Especifique el número de capas ocultas (densas), las neuronas por capa y la función de activación usada
- Especifique las clases de la capa de salida y la función de activación usada
- Especifique los parámetros por capa y el número total de parámetros entrenables del modelo
- Especifique las técnicas de regularización utilizadas (por ejemplo, Dropout, Batch Normalization o Data Augmentation)
- Posteriormente compile y entrene el modelo, especifique el optimizador, la función de pérdida y la métrica para monitorear la precisión

In [ ]:
# Responda aquí:

**5.** **[6 pts]** Evalúe el desempeño de su modelo, en términos de:<br>
- Precisión en entrenamiento y validación
- Tiempo de entrenamiento
- Gráfico de la función de pérdida y precisión
- Indique si hay sobreajuste o subajuste
- ¿Qué cambios podrían mejorar el modelo?

In [ ]:
# Responda aquí:

**6.** **[6 pts]** Hiperparámetros del entrenamiento<br>

Explique cómo afecta:
- Learning rate
- Batch size
- Número de épocas

Pruebe 2 o 3 combinaciones distintas y elija la más eficiente en términos de precisión y tiempo.

In [ ]:
# Responda aquí:

**7.** **[6 pts]** Predicciones y visualización de errores<br>

- Realice predicciones con su modelo entrenado.
- Visualice algunas imágenes clasificadas correctamente y otras mal clasificadas.
- Proponga hipótesis de por qué el modelo se equivocó.

In [ ]:
# Responda aquí: